In [45]:
import tensorflow as tf
from keras.models import load_model
from keras.layers import *
from keras import Model, Sequential, Input
from keras import optimizers
from keras.regularizers import l2
import keras.backend as K
import numpy as np
from pprint import pprint
import pickle


def load(filepath):
    with open(filepath, 'rb') as f:
        return pickle.load(f)

dataTriple = load('obj/triplets.pkl')
dataPairs = load('obj/pairs.pkl')
triplets = dataTriple['triplets'] / 256.
pairs = dataPairs['pairs'] / 256.
samePairs = load('obj/samePairs.pkl')
samePairs = samePairs['pairs'] / 256.

In [46]:
def get_darknet():
    model = load_model('darknet.h5')
    for layer in model.layers:
        layer.trainable = False
    return model

def make_style_model():
    darknet = get_darknet()
    
    # pretrained layers
    inputs = darknet.layers[0].input
    x = darknet.layers[16].output
    
    # new layers
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Lambda(lambda x: tf.nn.l2_normalize(x, axis=0))(x)
    base_model = Model(inputs, x)
    
    # siamese networks
    input_a = Input((256,256,3))
    input_b = Input((256,256,3))
    input_c = Input((256,256,3))
    
    encoding_a = base_model(input_a)
    encoding_b = base_model(input_b)
    encoding_c = base_model(input_c)
    
    dist_pos = Lambda(sqEucl, output_shape=(1,))([encoding_a, encoding_b])
    dist_neg = Lambda(sqEucl, output_shape=(1,))([encoding_a, encoding_c])
    triple_loss = Lambda(triplet_loss, output_shape=(1,))([dist_pos, dist_neg])
    
    triple_model = Model([input_a, input_b, input_c], triple_loss)
    pairModel = Model([input_a, input_b], dist_pos)
    
    return triple_model, pairModel

def triplet_loss(x):
    return K.maximum(x[0] - x[1] + 1, 0)

def sqEucl(x):
    return K.sum(K.square(x[0] - x[1]), axis=-1, keepdims=True)

def norm(x):
    return K.sqrt(sqEucl(x))
    
def identity(y_true, y_pred):
    return K.mean(K.square(y_pred))

In [ ]:
tripleModel, pairModel = make_style_model()

sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
tripleModel.compile(optimizer=sgd, loss=identity)
pairModel.compile(optimizer=sgd, loss=identity)
alphas = np.zeros(triplets.shape[0])


tripleModel.fit([triplets[:, 0], triplets[:, 1], triplets[:, 2]], alphas, epochs=2)


/Users/annab./anaconda3/lib/python3.6/site-packages/keras/models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Epoch 1/2
 256/1000 [======>.......................] - ETA: 1:14 - loss: 10.5169

In [ ]:

def predict(x, y):
    return pairModel.predict([x, y]).ravel()

for i in range(20):
    begin = i*20
    end = begin + 20
    predictions = predict(pairs[begin:end,0], pairs[begin:end,1]).reshape((-1,2))
    print(predictions.reshape((-1, 2)))

In [47]:
def get_darknet():
    model = load_model('darknet.h5')
    for layer in model.layers:
        layer.trainable = False
    return model

def make_content_model():
    darknet = get_darknet()
    
    # pretrained layers
    inputs = darknet.layers[0].input
    x = darknet.layers[16].output
    

    base_model = Model(inputs, x)
    
    # siamese networks
    input_a = Input((256,256,3))
    input_b = Input((256,256,3))

    
    encoding_a = base_model(input_a)
    encoding_b = base_model(input_b)

    
    dist_pos = Lambda(sqEucl, output_shape=(1,))([encoding_a, encoding_b])

    pairModel = Model([input_a, input_b], dist_pos)
    
    return  pairModel

def triplet_loss(x):
    return K.maximum(x[0] - x[1] + 1, 0)

def sqEucl(x):
    return K.sum(K.square(x[0] - x[1]), axis=-1, keepdims=True)

def norm(x):
    return K.sqrt(sqEucl(x))
    
def identity(y_true, y_pred):
    return K.mean(K.square(y_pred))

def content_loss_fail(p, x):
#      square difference of the activation function
    
    # N is the number of filters (at layer l).
#     N = p.shape[3]
#     # M is the height times the width of the feature map (at layer l).
#     M = p.shape[1] * p.shape[2]
    
#     K = (1 / (4 * N * M))
    _, h, w, d = p.get_shape()
    M = h.value * w.value
    N = d.value
    K = 1. / (2. * N**0.5 * M**0.5)
    loss = K * tf.reduce_sum(tf.pow((x - p), 2))
    return loss

# The content loss is the (scaled, squared) Euclidean distance
# between feature representations of the content and combination images.
def content_loss(content, combination):
    print(content)
    print(combination)
    return K.sum(K.square(combination - content))


In [48]:
 pairModel = make_content_model()

sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
# tripleModel.compile(optimizer=sgd, loss=content_loss)
pairModel.compile(optimizer=sgd, loss=content_loss)
alphas = np.zeros(triplets.shape[0])


# tripleModel.fit(, alphas, epochs=10)

/Users/annab./anaconda3/lib/python3.6/site-packages/keras/models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Tensor("lambda_50_target:0", shape=(?, ?), dtype=float32)
Tensor("lambda_50/Sum:0", shape=(?, 16, 16, 1), dtype=float32)


In [57]:
# Measure of closeness the left column(positive) should be less than right (negative)
print(pairs.shape)
print(samePairs.shape)
def predict(x, y):
    return pairModel.predict([x, y]).ravel()

for i in range(10):
   
    predictions = predict(samePairs[0:5,0], samePairs[5:10,1]).reshape((-1,2))
    print(predictions.reshape((-1, 2)))

(400, 2, 256, 256, 3)
(10, 2, 256, 256, 3)
[[ 183.45896912  418.7819519 ]
 [ 579.07019043  937.59570312]
 [ 724.72094727  313.10171509]
 ..., 
 [ 286.51123047  488.1796875 ]
 [ 451.91259766  479.30926514]
 [ 548.21258545  492.84646606]]
[[ 183.45896912  418.7819519 ]
 [ 579.07019043  937.59570312]
 [ 724.72094727  313.10171509]
 ..., 
 [ 286.51123047  488.1796875 ]
 [ 451.91259766  479.30926514]
 [ 548.21258545  492.84646606]]
[[ 183.45896912  418.7819519 ]
 [ 579.07019043  937.59570312]
 [ 724.72094727  313.10171509]
 ..., 
 [ 286.51123047  488.1796875 ]
 [ 451.91259766  479.30926514]
 [ 548.21258545  492.84646606]]
[[ 183.45896912  418.7819519 ]
 [ 579.07019043  937.59570312]
 [ 724.72094727  313.10171509]
 ..., 
 [ 286.51123047  488.1796875 ]
 [ 451.91259766  479.30926514]
 [ 548.21258545  492.84646606]]
[[ 183.45896912  418.7819519 ]
 [ 579.07019043  937.59570312]
 [ 724.72094727  313.10171509]
 ..., 
 [ 286.51123047  488.1796875 ]
 [ 451.91259766  479.30926514]
 [ 548.21258545  49